In [4]:
import torch
from torch import fx

class M(torch.nn.Module):
    def forward(self, x, y):
        return torch.add(x, y)

m_add = M()
x = y = torch.tensor([0])
torch.onnx.export(m_add, args=(x, y), f='temp.onnx')

In [5]:
import onnx
from tvm import relay

onnx_model = onnx.load("temp.onnx")
input_info = {'0': (1, 3, 299, 299), '1': (1, 3, 299, 299)}
mod, params = relay.frontend.from_onnx(onnx_model, input_info)

In [7]:
# %%script true
from tvm.relay import transform

mod = transform.InferType()(mod)
print(mod["main"].checked_type)

fn (Tensor[(1, 3, 299, 299), int64], Tensor[(1, 3, 299, 299), int64]) -> Tensor[(1, 3, 299, 299), int64]
